# Napari


## Setup: start the [desktop](../desktop)
The link should open in a different window. If you see an error message try refreshing the window.

Then run the first cell and wait for QT to be initialised before running the following cells. `Run all cells` will not work.

If you are running this locally you should see some log messages. If you are running this on mybinder wait a few seconds before running the Napari cell. Note adding `time.sleep(5)` to the cell does not work!

In [ ]:
# Use the novnc Desktop in the other window
import os
os.environ['DISPLAY'] = ':1.0'
# instantiate Qt GUI
%gui qt5

In [ ]:
from dask import delayed
import dask.array as da
import napari
from vispy.color import Colormap
import numpy
import omero
from omero.gateway import BlitzGateway
client = omero.client(host='ws://idr.openmicroscopy.org/omero-ws')
client.createSession('public', 'public')
conn = BlitzGateway(client_obj=client)
# conn = BlitzGateway(host='ws://idr.openmicroscopy.org/omero-ws',
#                    username='public', passwd='public', secure=True)
# conn.connect()

IMAGE_ID = 5514052
image = conn.getObject("Image", IMAGE_ID)

cache = {}

def get_lazy_stack(img, c=0):
    sz = img.getSizeZ()
    st = img.getSizeT()
    plane_names = ["%s,%s,%s" % (z, c, t) for t in range(st) for z in range(sz)]

    def get_plane(plane_name):
        if plane_name in cache:
            return cache[plane_name]
        z, c, t = [int(n) for n in plane_name.split(",")]
        pixels = img.getPrimaryPixels()
        p = pixels.getPlane(z, c, t)
        cache[plane_name] = p
        return p

    # read the first file to get the shape and dtype
    # ASSUMES THAT ALL FILES SHARE THE SAME SHAPE/TYPE
    sample = get_plane(plane_names[0])
    lazy_imread = delayed(get_plane)
    lazy_arrays = [lazy_imread(pn) for pn in plane_names]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
        for delayed_reader in lazy_arrays
    ]
    # Stack into one large dask.array
    if sz == 1 or st == 1:
        return da.stack(dask_arrays, axis=0)
    z_stacks = []
    for t in range(st):
        z_stacks.append(da.stack(dask_arrays[t * sz: (t + 1) * sz], axis=0))
    return da.stack(z_stacks, axis=0)
# create the viewer on the Desktop and display the image
with napari.gui_qt():
    viewer = napari.Viewer()
    for c, channel in enumerate(image.getChannels()):
        data = get_lazy_stack(image, c=c)
        start = channel.getWindowStart()
        end = channel.getWindowEnd()
        color = channel.getColor().getRGB()
        color = [r/256 for r in color]
        cmap = Colormap([[0, 0, 0], color])
        # Z-scale for 3D viewing
        z_scale = 1
        if image.getSizeZ() > 1:
            size_x = image.getPixelSizeX()
            size_z = image.getPixelSizeZ()
            if size_x is not None and size_z is not None:
                z_scale = [1, size_z / size_x, 1, 1]
        viewer.add_image(data, blending='additive',
                         contrast_limits=[start, end],
                         colormap=('from_omero', cmap),
                         # scale=[1, z_scale, 1, 1],
                         name=channel.getLabel())
conn.close()